# Quick introduction to Pinocchio (for kinematics)
This notebook is a quick introduction to [Pinocchio](https://github.com/stack-of-tasks/pinocchio), a very convenient library for robot kinematics and dynamics.

Pinocchio is a great library to compute kinematics and dynamics information of a robot. It is often used in algorithms, for example:
* to build a simulator (Pinocchio can deal with the kinematics and dynamics part of the simulation)
* in optimal control algorithms to help compute the robot dynamics and its derivatives
* in any algorithm that needs access to robot information (kinematics or dynamics)
* It also implements homogeneous transforms and all the associated algebra, quaternions, etc.

We use Meshcat to display the robot.

All the robots are described by a URDF file (which describes their kinematic tree).



## Requirement
To run these examples you will need to install these two packages:
* [Pinocchio](https://github.com/stack-of-tasks/pinocchio), you can install it directly through conda `conda install pinocchio -c conda-forge` or follow the [installation instructions](https://stack-of-tasks.github.io/pinocchio/download.html#Install_3)
* [Meshcat](https://github.com/rdeits/meshcat-python) which is used for the visualization of the robot. You can install it directly through pip `pip install meshcat`

In [ ]:
import numpy as np
import pinocchio
import os
import time
import meshcat
import matplotlib.pyplot as plt

from IPython.display import clear_output

# be careful we will only print the first 4 digits and round small numbers in arrays
np.set_printoptions(suppress=True, precision=4)

# Kuka iiwa robot

## Create a Pinocchio model and display the robot with Meshcat

In [ ]:
# the directory where the robot models are located
package_dirs = './urdf/'

# the name of the URDF model (the robot model)
urdf_file = 'iiwa.urdf'
END_EFF_FRAME_ID = 17 # number of the frame corresponding to the end-effector

# we load the urdf models with Pinocchio (it will build a robot model, its kinematic tree, etc)
urdf = package_dirs + urdf_file
robot = pinocchio.RobotWrapper.BuildFromURDF(urdf, package_dirs)

# we create the visualization for Meshcat
viz = pinocchio.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install Python meshcat")
    print(err)
    sys.exit(0)
viz.loadViewerModel()

q = pinocchio.neutral(robot.model)
viz.display(q) # to update the meshcat display

## Forward kinematics

In [ ]:
# we get the joint angles of the robot for its default position and display the robot
q = pinocchio.neutral(robot.model)
viz.display(q) # to update the meshcat display

print(f'the default joint angle positions are {q}\n\n')

# we can ask Pinocchio to compute the forward kinematics
robot.forwardKinematics(q)

# we read the position of the end-effector frame
T_S_F = robot.framePlacement(q, END_EFF_FRAME_ID)

print(f'the pose (homogenous transform) of the end-effector frame is\n {T_S_F.homogeneous}\n\n')

# we can also extract the individual rotation and translation
print(f'the orientation is\n R = {T_S_F.rotation}\n\n and the position is\n p = {T_S_F.translation}')

In [ ]:
# we can chose other angles for the joints
q = np.array([1., 0., 1., 2., -2., 2., 0.])
viz.display(q) # to update the meshcat display

print(f'the joint angle positions are {q}\n\n')

# we can ask Pinocchio to compute the forward kinematics
robot.forwardKinematics(q)

# we read the position of the end-effector frame
T_S_F = robot.framePlacement(q, END_EFF_FRAME_ID)

print(f'the pose (homogenous transform) of the end-effector frame is\n {T_S_F.homogeneous}\n\n')

# we can also extract the individual rotation and translation
print(f'the orientation is\n R = {T_S_F.rotation}\n\n and the position is\n p = {T_S_F.translation}')

## comparing with our own forward kinematics
We can compare this with our own forward kinematics function (cf. FK notebook for more details)

In [ ]:
def translate(vector):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[0:3,3] = vector

    ### we return the object
    return transform

def rotateX(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[1:3,1:3] = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle), np.cos(angle)]])

    ### we return the object
    return transform

def rotateY(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[0,0] = np.cos(angle)
    transform[0,2] = np.sin(angle)
    transform[2,0] = -np.sin(angle)
    transform[2,2] = np.cos(angle)

    ### we return the object
    return transform

def rotateZ(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    
    # here you can fill the rest of the transform
    transform[0:2,0:2] = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle), np.cos(angle)]])

    
    ### we return the object
    return transform

def forward_kinematics_KUKA(theta):
    T_BJ1 = translate([0,0,0.1575]) @ rotateZ(theta[0])
    T_J1J2 = translate([0,0,0.2025]) @ rotateY(theta[1])
    T_J2J3 = translate([0,0,0.2045]) @ rotateZ(theta[2])
    T_J3J4 = translate([0,0,0.2155]) @ rotateY(-theta[3])
    T_J4J5 = translate([0,0,0.1845]) @ rotateZ(theta[4])
    T_J5J6 = translate([0,-0.0607, 0.2155]) @rotateY(theta[5])
    T_J6J7 = translate([0,0.0607, 0.0810])@ rotateZ(theta[6])
    T_J7E = translate([0,0,0.04])

    T_SF = T_BJ1 @ T_J1J2 @ T_J2J3 @ T_J3J4 @ T_J4J5 @ T_J5J6 @ T_J6J7 @ T_J7E
    
    ### we return the pose of the end-effector
    return T_SF

T_FK_custom = forward_kinematics_KUKA(q)

print(f'for joint configuration {q}')
print(f'FK from Pinocchio give a rotation of\n {T_S_F.rotation} and a translation of {T_S_F.translation}')
print(f'our own FK gives a rotation of\n {T_FK_custom[0:3,0:3]} and a translation of {T_FK_custom[0:3,3]}')

## a simple kinematics simulator

In [ ]:
# we can move the joints using sine functions and display them to build our first "simulator"

t = 0.0
dt = 0.01
print_count = 1
print_every = 10

while t < 10.:
    # we can chose other angles for the joints
    q = np.array([1., 0., 1., 2., -2., 2., 0.]) * np.sin(2*np.pi*t)
    viz.display(q) # to update the meshcat display

    # we can ask Pinocchio to compute the forward kinematics
    robot.forwardKinematics(q)

    # we read the position of the end-effector frame
    T_S_F = robot.framePlacement(q, END_EFF_FRAME_ID)

    if print_count >= print_every:
        print_count = 1
        print(f'time = {t:0,.2f}\n\n')
        print(f'the joint angle positions are {q}\n\n')
        print(f'the pose (homogenous transform) of the end-effector frame is\n {T_S_F.homogeneous}\n\n')
        # we can also extract the individual rotation and translation
        print(f'the orientation is\n R = {T_S_F.rotation}\n\n and the position is\n p = {T_S_F.translation}')
        clear_output(wait=True)
    else:
        print_count += 1
    
    t += dt
    time.sleep(dt)



# Talos Humanoid robot

We can also use more complex robots and easily access all their kinematic information. Below is an example with the Talos Humanoid robot.

In [ ]:
# the directory where the robot models are located
package_dirs = './urdf/'

# the name of the URDF model (the robot model)
urdf_file = 'talos_reduced.urdf'

# the number of the frames corresponding to limbs of interest (as exposed in Pinocchio)
BASE_LINK_ID = 2
LEFT_FOOT_ID = 16
RIGHT_FOOT_ID = 30
LEFT_HAND_ID = 56
RIGHT_HAND_ID = 90
HEAD_ID = 106

# we load the urdf models with Pinocchio
urdf = package_dirs + urdf_file
robot = pinocchio.RobotWrapper.BuildFromURDF(urdf, package_dirs, root_joint=pinocchio.JointModelFreeFlyer())

# we create the visualization
viz = pinocchio.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)

try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install Python meshcat")
    print(err)
    sys.exit(0)
    
viz.loadViewerModel()


# place the robot in its default position and display it
q0 = pinocchio.neutral(robot.model)
viz.display(q0)


## understanding the configuration space of the humanoid
A humanoid robot is not attached to the ground, we thus need to describe its orientation and position with respect to a frame attached to the ground. Hence we need 6 more degrees of freedom that describe the "floating base" of the robot, i.e. the "base frame" of the robot is a frame attached on the robot that allows to describe the pose of the robot with respect to a refrence frame attached to the ground.

In Pinocchio, the pose of "floating-base" robots which are not attached to the ground is described using a translation and a quaternion for the rotation. Hence the first 3 numbers of the robot configuration is the translation vector and the next 4 numbers the quaternion (the orientation of the robot w.r.t. a spatial frame of reference).

In [ ]:
# we get the configuration of the robot for its default position and display the robot
q = pinocchio.neutral(robot.model)
viz.display(q) # to update the meshcat display

print(f'the default configuration of the robot is\n {q}\n\n')
print(f'WARNING the first 3 numbers of q is the position of the robot in space and the next 4 numbers are the quaternion describing its orientation\n\n')

# the pose of the robot in space is
print(f'we can easily transform the quaternion into a rotation matrix')
T_base = pinocchio.XYZQUATToSE3(q[0:7])
print(f'the position and orientation of the robot in space is then\n {T_base} \n\n')

# the joint configuration
q_joint = q[7:]
print(f'and the joint positions (angles) are\n {q_joint} \n\n')

In [ ]:
# we translate and rotate the robot in space by applying a body (right multiply) or spatial (left multiply) transformation
# pinocchio provides tools to handle quaternions and change them into rotation matrices
# it actually provides all the tools we need to handle homogeneous transforms, etc

# this function allows to transform the position + quaternion part into an homogeneous transform
T_base = pinocchio.XYZQUATToSE3(q[0:7])

print('the pose of the robot is')
print(T_base)

# we can then apply a rotation on the robot, for example a rotation around X of pi/2
rot = pinocchio.AngleAxis(np.pi/2., np.array([1.,0.,0.])).matrix()
print(f'this is a rotation of pi/2 around x:\n {rot}')

# we can put this into an homogeneous transform
T = pinocchio.SE3.Identity() # get an identity transform
T.rotation = rot

# we can also apply a translation along the body-y direction
T.translation = np.array([0., 1.0, 0.])

# we can apply this transformation to the base of the robot
# we do a body transformation (multiply on the right side) the function act multiplies 2 homogeneous transforms
T_base_new = T_base.act(T)
print(f'the new pose of the robot will be \n {T_base_new} \n\n')

# we can then transform this into pos + quaternion and update the configuration q of the robot
q[0:7] = pinocchio.SE3ToXYZQUAT(T_base_new)
viz.display(q)

## back to forward kinematics

In [ ]:
# we get the joint angles of the robot for its default position and display the robot
q = pinocchio.neutral(robot.model)
viz.display(q) # to update the meshcat display

# we can ask Pinocchio to compute the forward kinematics
robot.forwardKinematics(q)

# we read the position of the feet and hands
T_right_foot = robot.framePlacement(q, RIGHT_FOOT_ID)
T_left_foot = robot.framePlacement(q, LEFT_FOOT_ID)
T_right_hand = robot.framePlacement(q, RIGHT_HAND_ID)
T_left_hand = robot.framePlacement(q, LEFT_HAND_ID)

print(f'the pose of the right foot frame is is\n {T_right_foot.homogeneous}\n\n')
print(f'the pose of the left foot frame is is\n {T_left_foot.homogeneous}\n\n')
print(f'the pose of the right hand frame is is\n {T_right_hand.homogeneous}\n\n')
print(f'the pose of the left hand frame is is\n {T_left_hand.homogeneous}\n\n')


# the position of the right hand in the world frame is
p_rh = T_right_hand.translation
# the position of the left hand in the world frame is
p_lh = T_left_hand.translation

print(f'the distance between the hands is {np.linalg.norm(p_rh - p_lh)}\n\n')

print(f'the position of the right hand with respect to the left hand is\n {T_left_hand.inverse().act(T_right_hand).homogeneous}\n\n')
print(f'the position of the left hand with respect to the right hand is\n {T_right_hand.inverse().act(T_left_hand).homogeneous}\n\n')
print('we verify that they are indeed at the proper distance apart')

In [ ]:
# we can move the joints using sine functions and display them to build our first "simulator"

T = 10.
dt = 0.01
print_count = 0
print_every = 10

t = np.linspace(0., T, int(T/dt)+1)

feet_dist = np.zeros([t.size])

for i in range(t.size):
    # we can chose other angles for the joints
    q = np.array([0., 0., 0., 0., 0., 0., 1., 0., 0., -1., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., -.5, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]) * np.sin(2. * np.pi * t[i])
    viz.display(q) # to update the meshcat display

    # we can ask Pinocchio to compute the forward kinematics
    robot.forwardKinematics(q)

    # we read the position of the end-effector frame
    T_right_foot = robot.framePlacement(q, RIGHT_FOOT_ID)
    T_left_foot = robot.framePlacement(q, LEFT_FOOT_ID)

    # the position of the right hand in the world frame is
    p_rf = T_right_foot.translation
    # the position of the left hand in the world frame is
    p_lf = T_left_foot.translation
        
    feet_dist[i] = np.linalg.norm(p_rf - p_lf, ord=2)
    
    if print_count >= print_every:
        print_count = 0
        print(f'time = {t[i]:0,.2f}\n\n')
        print(f'the position of the right foot frame is {T_right_foot.translation}')
        print(f'the position of the left foot frame is {T_left_foot.translation}')
        print(f'the distance between the feet is {feet_dist[i]}\n\n')
        clear_output(wait=True)
    else:
        print_count += 1
    
    time.sleep(dt)



In [ ]:
plt.figure()
plt.plot(t, feet_dist)
plt.title('Distance between the feet as a function of time')